# Access BDC cubes

### install EOCubes

##### (uncomment the line if you need to install the package)

In [ ]:
#pip install --user git+https://github.com/brazil-data-cube/EOCubes@master --upgrade --no-cache-dir

### Import EOCubes

In [ ]:
from bdc_eocubes.business import Business as eocubes

### List all existing cubes

In [ ]:
cubes = eocubes.list_cubes()
cubes

### Get cube description

In [ ]:
cube_s10 = eocubes.describe_cube('S10m:STACK')
cube_s10

### Get cube

In [ ]:
my_cube = eocubes.get_cube('S10m:STACK', bbox= '-46.62597656250001,-13.19716452328198,-45.03570556640626,-12.297068292853805')
my_cube

### Print all available Bands for this cube

In [ ]:
bands = my_cube['2018-09-01/2019-08-01']['tiles']['089098']['bands'].keys()
print( bands )

### Given the nir band, print all available Bands for this cube

In [ ]:
links_nir = my_cube['2018-09-01/2019-08-01']['tiles']['089098']['bands']['nir']
links_nir

### Print first image link

In [ ]:
filename_nir = links_nir[0]
filename_nir

### Imports

In [ ]:
#### Import Libs ####
#import numpy
import rasterio

from math import floor, ceil
from matplotlib import pyplot as plt
from pyproj import Proj
from rasterio.warp import transform
from rasterio.windows import Window

### Function to crop dataset with longitude and latitude 

In [ ]:
def longlat2window(lon, lat, dataset):
    """
    Args:
        lon (tuple): Tuple of min and max lon
        lat (tuple): Tuple of min and max lat
        dataset: Rasterio dataset

    Returns:
        rasterio.windows.Window
    """
    p = Proj(dataset.crs)
    t = dataset.transform
    xmin, ymin = p(lon[0], lat[0])
    xmax, ymax = p(lon[1], lat[1])
    col_min, row_min = ~t * (xmin, ymin)
    col_max, row_max = ~t * (xmax, ymax)
    return Window.from_slices(rows=(floor(row_max), ceil(row_min)),
                              cols=(floor(col_min), ceil(col_max)))

### Load image using Row Col

In [ ]:
with rasterio.open(filename_nir) as dataset:
    print(dataset.profile)
    #img = dataset.read(1) ### open entire scene is heavy
    
    img = dataset.read(1, window=Window(0, 0, 3000, 3000)) ### Window(col_off, row_off, width, height)
    
    plt.imshow(img, cmap='gray')
    plt.show()

### Load image using Bounding box 

In [ ]:
### CREATING BBOX ###
w = -45.90
n = -12.6
e = -45.40
s = -12.90

bbox = ( (w,e), (s,n) )

with rasterio.open(filename_nir) as dataset:
    img = dataset.read(1, window = longlat2window(bbox[0], bbox[1], dataset)) ### Window(col_off, row_off, width, height)

    plt.imshow(img, cmap='gray')
    plt.show()

### select a few bands

In [ ]:
my_bands = ('blue', 'green', 'red', 'nir')
my_bands

### Import Xarray lib

In [ ]:
import xarray

### Load entire image as Xarray

In [ ]:
nir = xarray.open_rasterio(filename_nir)
nir.variable.data

### Functions to load cubes as xarray dataset

In [ ]:
import re 

def file_to_da(filename, bbox):
    with rasterio.open(filename) as dataset:
        img = dataset.read(1, window = longlat2window(bbox[0], bbox[1], dataset)) 
        
    da = xarray.DataArray(img)
    #da = xarray.open_rasterio(img)

    match = re.findall(r'\d{4}-\d{2}-\d{2}', filename)[-1]
    da.coords['time'] = match

    return da

def img2xarray(band, bbox):
    links = cube[band]
    #load bands to xarray dataset
    list_of_data_arrays=[file_to_da(link, bbox) for link in links]
        
    combined = xarray.concat(list_of_data_arrays, dim='time')
        
    return combined

### Set a new Bounding box, select a few bands and load all to Xarray Dataset

In [ ]:
w = -45.9
n = -12.6
e = -45.8
s = -12.7
bbox = ( (w,e), (s,n) )
my_bands = ( 'red', 'nir' )

cube_info = my_cube['2018-09-01/2019-08-01']['tiles']['089098']['bands']

xray_dataset = [img2xarray(band, bbox) for band in my_bands]

cube_dataset = xarray.concat(xray_dataset, dim = 'bands')

In [ ]:
cube_dataset

### Calculating NDVI

### Plot

### LEAFLET

In [ ]:
from ipyleaflet import Map, WMSLayer

In [ ]:
m = Map(center=[-12.75, -45.65], zoom=10)
m

In [ ]:
#sh_wms_url = 'https://services.sentinel-hub.com/ogc/wms/' + INSTANCE_ID + '?showlogo=0&time=2017-07-18/2017-07-18'
#sh_wms_url

In [ ]:
#m.add_layer(WMSLayer(url=sh_wms_url, layers="GEOLOGY", tile_size=512))
#print(m)

In [ ]:
#center coordinates resulting in a bit off-center
m.zoom = 8
m.center=[-12.6, -46] 

In [ ]:
m.bounds